# 3. Preprocessing - Source Data Augmentation and Sampling

Last Updated: December 11, 2018

In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import SVG
from sklearn import preprocessing
import os
from os.path import join, dirname
from os.path import dirname, join
from tqdm import tqdm_notebook
import pandas as pd
from pandas import pivot
from pandas import tools
import numpy as np
import cv2
import re
import io
import sys 
import pip
import csv
from sklearn.model_selection import train_test_split
import glob
import random
import warnings
from pandas import DataFrame   
import pickle
import cloudpickle
import azureml
import string
from scipy import stats
import pyparsing
import tempfile  
import logging
import graphviz

### Working Directory Definition

In [2]:
os.chdir('D://MAFData')
base_dir = 'D://MAFData'
WorkingDir = os.path.join(base_dir, 'Workingdir')

print(base_dir)
os.getcwd()

D://MAFData


'D:\\MAFData'

### Read in Cleaned Source Data

In [3]:
dtypes = {'Account_Number': object,
                            'MISDATE': str,
                            'ATTRITION_FLAG':                            object,
                            'ACTUAL_CURRENT_BALANCE':                     float,
                              'ACCOUNT_CREATION_DATE':                   object,
                            'Credit_Limit':                               int,
                            'Last_Statement_Closing_Balance':             int,
                            'Last_Statement_Date':                       str,
                            'Last_Statement_Minimum_Due':                 int,
                            'Monthly_Salary':                            int,
                            'Months_On_Book':                             int,
                            'UAE_Citizen':                                int,
                            'Next_Statement_Date':                        object,
                            'LAST_ACTIVITY_DATE':                        object,
                            'No_Of_Due_Stmt':                            int,
                            'Product':                                    int,
                            'Predicted_Date':                            object,                            
                            'Fee_Paying':                                int,
                            'Non_Fee':                                   int,
                            'Staff_Fee':                                 int,
                            '1st_yr_Fre':                                int,
                            'APR_Rate_Low':                              int,
                            'APR_Rate_Med':                              int,
                            'APR_Rate_High':                             int,
                            'Prod_Cat_Voyager':                          int,
                            'Prod_Cat_Cashback':                         int,
                            'Perc_Credit_Limit_Used':                   float,
                            'Days_on_Book':                               int,
                            'Days_from_Last_activity':                  int,
                                    'Incident_Category':                 str,
                            'Incident_Type':                      str,
                            'Incident_Cat_Sequence_Daily':        str,
                            'Incident_Type_Sequence_Daily':        str,
                            'Billing_Amount_Sum_Payment':       float, 
                            'Billing_Amount_Sum_Spend_ESS':     float,
                            'Billing_Amount_Sum_Spend_NON_ESS': float, 
                            'Billing_Amount_Spend_CASH':        float,
                            'Billing_Amount_Violation_Fee':     float, 
                            'Transaction_Count_Payment':        float,
                            'Transaction_Count_Spend_ESS':     float, 
                            'Transaction_Count_Spend_NON_ESS': float,
                            'Transaction_Count_Spend_CASH':    float, 
                            'Transaction_Count_Foreclosure':    float,
                            'Transaction_Count_BT':            float, 
                            'Transaction_Count_LOP':           float,
                            'Transaction_Count_EPP':           float, 
                            'Transaction_Count_Violation_Fee': float,
                            'Timestep':                                   int}
parse_dates = ['MISDATE','Last_Statement_Date','LAST_ACTIVITY_DATE', 'ACCOUNT_CREATION_DATE']
df1 = pd.read_csv('Merged_cleaned2.csv',sep=',',encoding='utf-8', header = 0,  
                  dtype = dtypes,parse_dates=parse_dates)
df1.head(2)

,Unnamed: 0,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,...,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee,Timestep
0,0,MAFF_ACCT_0000002125,2017-09-21,OPEN,2012-10-24,10483.11,18000,2017-09-21,17391,2017-09-10,...,0.0,1,0,0,0.0,0,0,0,0,1
1,1,MAFF_ACCT_0000002125,2017-09-20,OPEN,2012-10-24,9858.10,18000,2017-09-20,17391,2017-09-10,...,0.0,1,1,0,0.0,0,0,0,0,2


In [4]:
del df1['Unnamed: 0']
df1.shape

(10544170, 51)

### Create Label Index

In [7]:
Class_Distribution_check = df1.sort_values(['Account_Number','MISDATE'], ascending=[True, False])
Class_Distribution_check = Class_Distribution_check.reset_index(drop=True)
Class_Distribution_check = Class_Distribution_check.groupby(['Account_Number']).head(1)
Class_Distribution_check = Class_Distribution_check.reset_index(drop=True)
Class_Distribution_check = pd.DataFrame(Class_Distribution_check)
Class_Distribution_check = Class_Distribution_check.reset_index(drop=True)
Class_Distribution_check['ATTRITION_FLAG'].value_counts()

OPEN         74267
VOLUNTARY     6842
Name: ATTRITION_FLAG, dtype: int64

In [8]:
Attrits = Class_Distribution_check[(Class_Distribution_check['ATTRITION_FLAG'] == 'VOLUNTARY')]
Attrits['date'] = pd.to_datetime(Attrits['MISDATE'], format='%Y-%m-%d')
#Attrits['ATTRITION_FLAG'].groupby([Attrits['date'].dt.year, Attrits['date'].dt.week]).count().plot(kind="bar", figsize=(25,15))

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [9]:
Attrits.to_csv('Attrits.csv',sep=',',encoding='utf-8')

In [11]:
Labels = df1.sort_values(['Account_Number','MISDATE'], ascending=[True, False])
Labels = Labels.groupby(['Account_Number']).head(1)

Labels1 = pd.DataFrame(Labels.groupby(['Account_Number'])['ATTRITION_FLAG'].apply(' '.join))
Labels1.loc[Labels1['ATTRITION_FLAG'].str.contains('VOLUNTARY'), 'ATTRITION_FLAG'] = 'VOLUNTARY'
Labels1.loc[Labels1['ATTRITION_FLAG'].str.contains('OPEN'), 'ATTRITION_FLAG'] = 'OPEN'

Labels = pd.DataFrame(Labels1)
Labels = Labels.reset_index()
Labels = Labels[['Account_Number','ATTRITION_FLAG']]
Labels['ATTRITION_FLAG'].value_counts()
Labels.head(1)

OPEN         74267
VOLUNTARY     6842
Name: ATTRITION_FLAG, dtype: int64

,Account_Number,ATTRITION_FLAG
0,MAFF_ACCT_0000002125,OPEN


### Write out Account Attrition Labels

In [12]:
Labels.to_csv('Account_Churn_Labels.csv',sep=',',encoding='utf-8')

### Remove Predictive Period

In [13]:
#in this case we have a 130 timestep dataset.  Removing latest 14 days and renumbering the timesteps.
df1 = df1.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
df1 = df1.reset_index(drop=True)
df1 = df1.groupby('Account_Number').tail(116)
df1 = pd.DataFrame(df1)
del df1['ATTRITION_FLAG']
df1.shape
df1.head(1)

(9408644, 50)

,Account_Number,MISDATE,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,Monthly_Salary,...,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee,Timestep
14,MAFF_ACCT_0000002125,2017-09-07,2012-10-24,17245.45,18000,2017-09-07,4490,2017-08-10,224,9000,...,0.0,1,0,0,0.0,0,0,0,0,15


### Rewrite Timestep

In [14]:
df1 = df1.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
df1 = df1.reset_index(drop=True)
df1['Timestep'] = df1.groupby(['Account_Number']).cumcount()+1
df1.shape
df1.head(1)

(9408644, 50)

,Account_Number,MISDATE,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,Monthly_Salary,...,Transaction_Count_Payment,Tranasaction_Count_Spend_ESS,Tranasaction_Count_Spend_NON_ESS,Tranasaction_Count_Spend_CASH,Transaction_Count_Foreclosure,Tranasaction_Count_BT,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee,Timestep
0,MAFF_ACCT_0000002125,2017-09-07,2012-10-24,17245.45,18000,2017-09-07,4490,2017-08-10,224,9000,...,0.0,1,0,0,0.0,0,0,0,0,1


### Add Trend Variables

In [15]:
#Adding after clipping predictive period to avoid information leakage.
df1.columns

Index(['Account_Number', 'MISDATE', 'ACCOUNT_CREATION_DATE',
       'ACTUAL_CURRENT_BALANCE', 'Credit_Limit', 'LAST_ACTIVITY_DATE',
       'Last_Statement_Closing_Balance', 'Last_Statement_Date',
       'Last_Statement_Minimum_Due', 'Monthly_Salary', 'Months_On_Book',
       'Next_Statement_Date', 'Monthly_Salary.1', 'Next_Statement_Date.1',
       'No_Of_Due_Stmt', 'Product', 'Product_Fee_Type', 'Days_On_Book',
       'Days_From_Statement', 'Days_from_Last_Activity', 'Incident_Category',
       'Incident_Type', 'Incident_Cat_Sequence_Daily',
       'Incident_Type_Sequence_Daily', 'Incident_Num', 'APR_Rate_Low',
       'APR_Rate_Med', 'APR_Rate_High', 'Prod_Cat_Voyager',
       'Prod_Cat_Cashback', 'Nationality_Cat', 'Perc_Credit_Limit_Used',
       'POINTS_REDEEMED', 'POINTS_BALANCE_CLOSING', 'POINTS_EARNED',
       'Billing_Amount_Sum_Payment', 'Billing_Amount_Sum_Spend_ESS',
       'Billing_Amount_Sum_Spend_NON_ESS', 'Billing_Amount_Spend_CASH',
       'Billing_Amount_Violation_Fee'

### Create Rolling Balance Gradients Features

In [16]:
df1['Balance_Gradient_window_14'] = np.gradient(df1['ACTUAL_CURRENT_BALANCE'].rolling(center=False,window=14).mean()).astype('float')
df1['Balance_Gradient_window_28'] = np.gradient(df1['ACTUAL_CURRENT_BALANCE'].rolling(center=False,window=28).mean()).astype('float')
df1['Balance_Gradient_window_74'] = np.gradient(df1['ACTUAL_CURRENT_BALANCE'].rolling(center=False,window=74).mean()).astype('float')
df1['Balance_Gradient_window_14']=df1['Balance_Gradient_window_14'].replace(-np.nan, 0).replace(np.nan, 0).round(5)
df1['Balance_Gradient_window_28']=df1['Balance_Gradient_window_28'].replace(-np.nan, 0).replace(np.nan, 0).round(5)
df1['Balance_Gradient_window_74']=df1['Balance_Gradient_window_74'].replace(-np.nan, 0).replace(np.nan, 0).round(5)
df1['Perc_Used_Gradient_window_14'] = np.gradient(df1['Perc_Credit_Limit_Used'].rolling(center=False,window=14).mean()).astype('float')
df1['Perc_Used_Gradient_window_28'] = np.gradient(df1['Perc_Credit_Limit_Used'].rolling(center=False,window=28).mean()).astype('float')
df1['Perc_Used_Gradient_window_74'] = np.gradient(df1['Perc_Credit_Limit_Used'].rolling(center=False,window=74).mean()).astype('float')
df1['Perc_Used_Gradient_window_14']=df1['Perc_Used_Gradient_window_14'].replace(-np.nan, 0).replace(np.nan, 0).round(5)
df1['Perc_Used_Gradient_window_28']=df1['Perc_Used_Gradient_window_28'].replace(-np.nan, 0).replace(np.nan, 0).round(5)
df1['Perc_Used_Gradient_window_74']=df1['Perc_Used_Gradient_window_74'].replace(-np.nan, 0).replace(np.nan, 0).round(5)
df1.tail(2)

,Account_Number,MISDATE,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,Monthly_Salary,...,Tranasaction_Count_LOP,Tranasaction_Count_EPP,Tranasaction_Count_Violation_Fee,Timestep,Balance_Gradient_window_14,Balance_Gradient_window_28,Balance_Gradient_window_74,Perc_Used_Gradient_window_14,Perc_Used_Gradient_window_28,Perc_Used_Gradient_window_74
9408642,MAFF_ACCT_0000266356,2017-01-14,2016-09-29,-962.0,30000,2017-01-02,-962,2017-01-05,0,700000,...,0,0,0,115,-657.64286,-328.82143,-135.00405,-0.02192,-0.01096,-0.0045
9408643,MAFF_ACCT_0000266356,2017-01-13,2016-09-29,-962.0,30000,2017-01-02,-962,2017-01-05,0,700000,...,0,0,0,116,-657.64286,-328.82143,-135.00405,-0.02192,-0.01096,-0.0045


In [17]:
df1.shape

(9408644, 56)

In [18]:
df1 = df1.reset_index(drop=True)
df1.columns

Index(['Account_Number', 'MISDATE', 'ACCOUNT_CREATION_DATE',
       'ACTUAL_CURRENT_BALANCE', 'Credit_Limit', 'LAST_ACTIVITY_DATE',
       'Last_Statement_Closing_Balance', 'Last_Statement_Date',
       'Last_Statement_Minimum_Due', 'Monthly_Salary', 'Months_On_Book',
       'Next_Statement_Date', 'Monthly_Salary.1', 'Next_Statement_Date.1',
       'No_Of_Due_Stmt', 'Product', 'Product_Fee_Type', 'Days_On_Book',
       'Days_From_Statement', 'Days_from_Last_Activity', 'Incident_Category',
       'Incident_Type', 'Incident_Cat_Sequence_Daily',
       'Incident_Type_Sequence_Daily', 'Incident_Num', 'APR_Rate_Low',
       'APR_Rate_Med', 'APR_Rate_High', 'Prod_Cat_Voyager',
       'Prod_Cat_Cashback', 'Nationality_Cat', 'Perc_Credit_Limit_Used',
       'POINTS_REDEEMED', 'POINTS_BALANCE_CLOSING', 'POINTS_EARNED',
       'Billing_Amount_Sum_Payment', 'Billing_Amount_Sum_Spend_ESS',
       'Billing_Amount_Sum_Spend_NON_ESS', 'Billing_Amount_Spend_CASH',
       'Billing_Amount_Violation_Fee'

### Save Local Copy of Entire Data File

In [20]:
#Save intermediate working file.
df1 = pd.DataFrame(df1)
df1.to_csv('Complete_Prepped_DataFrame.csv',sep=',',encoding='utf-8')

## Seperate Text and Categorical Series with Augmentations and Save File

In [21]:
df_Categorical = df1[['Account_Number', 'Days_On_Book', 'Days_From_Statement', 'Days_from_Last_Activity','Incident_Num','Incident_Category', 'Incident_Type', 'Incident_Cat_Sequence_Daily',
       'Incident_Type_Sequence_Daily','Perc_Credit_Limit_Used','MISDATE' ]]
df_Categorical.columns

Index(['Account_Number', 'Days_On_Book', 'Days_From_Statement',
       'Days_from_Last_Activity', 'Incident_Num', 'Incident_Category',
       'Incident_Type', 'Incident_Cat_Sequence_Daily',
       'Incident_Type_Sequence_Daily', 'Perc_Credit_Limit_Used', 'MISDATE'],
      dtype='object')

In [ ]:
#Save intermediate working file.
df_Categorical = pd.DataFrame(df_Categorical)
df_Categorical.to_csv('Categorical_dataframe.csv',sep=',',encoding='utf-8')

### Seperate Numerical Series and Save File

In [ ]:
df1['Billing_Amount_Sum_Payment']=df1['Billing_Amount_Sum_Payment'].replace(-np.nan, 0).replace(np.nan, 0)        
#df1.isnull().sum()

In [ ]:
df1.columns

In [ ]:
df_Numerical = df1[[
    'Account_Number', 'MISDATE',  
       'ACTUAL_CURRENT_BALANCE', 'Credit_Limit',
       'Last_Statement_Closing_Balance', 
       'Last_Statement_Minimum_Due', 'Monthly_Salary', 'Months_On_Book',
        'No_Of_Due_Stmt',
       'Days_On_Book', 'Days_From_Statement', 'Days_from_Last_Activity',
    
    
       'Incident_Num', 'APR_Rate_Low',
       'APR_Rate_Med', 'APR_Rate_High', 'Prod_Cat_Voyager',
       'Prod_Cat_Cashback', 'Nationality_Cat', 'Perc_Credit_Limit_Used',
       'POINTS_REDEEMED', 'POINTS_BALANCE_CLOSING', 'POINTS_EARNED',
       'Billing_Amount_Sum_Payment', 'Billing_Amount_Sum_Spend_ESS',
       'Billing_Amount_Sum_Spend_NON_ESS', 'Billing_Amount_Spend_CASH',
       'Billing_Amount_Violation_Fee', 'Transaction_Count_Payment',
       'Transaction_Count_Spend_ESS', 'Transaction_Count_Spend_NON_ESS',
       'Transaction_Count_Spend_CASH', 'Transaction_Count_Foreclosure',
       'Transaction_Count_BT', 'Transaction_Count_LOP',
       'Transaction_Count_EPP', 'Transaction_Count_Violation_Fee',
       'Timestep', 'Balance_Gradient_window_14', 'Balance_Gradient_window_28',
       'Balance_Gradient_window_74', 'Perc_Used_Gradient_window_14',
       'Perc_Used_Gradient_window_28', 'Perc_Used_Gradient_window_74'
]]




In [ ]:
df_Numerical.shape
df_Numerical.dtypes


In [ ]:
df_Numerical = pd.DataFrame(df_Numerical)
df_Numerical.head(4)

In [ ]:
df_Numerical.to_csv('Numerical_dataframe.csv',sep=',',encoding='utf-8')

### End -  Prepped Data Frames Script